In [8]:
import os
import itertools
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import cross_val_score,cross_val_predict
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

### Funcrion to load data at individual position 

In [11]:
def encode_data_per_position(fileprefix,
                data_dir="../../../DATA/02_L2MLR",
                features=("1mer",), 
                w=0,
                i=1):
    
    try:
        ohe_file = os.path.join(data_dir, "ROHS_MWORDS_1mer", f"{fileprefix}_1mer.tsv")
        
        
        ohe_df = pd.read_csv(ohe_file, header=None, sep="\t")
        ohe_data = {row[0]: {'aff': float(row[0].split('__')[1]), 
                             'feat': row.iloc[1:].tolist()} for _, row in ohe_df.iterrows()}
        
        print(f"1mer data loaded from {ohe_file}... \n")
        
        
    except FileNotFoundError as fe:
        raise FileNotFoundError(f"File {ohe_file} Not found: {fe}")
    
    combined_data = ohe_data.copy()
    
    
    if isinstance(features, tuple) and len(features) > 1:
        
        print(f"Will load the additional flex feature {features[1]}")
        try:
            
            flex_feat_file = os.path.join(data_dir, "ROHS_MWORDS_FLEX", f"{fileprefix}_{features[1]}_{w}.tsv") 
            flex_df = pd.read_csv(flex_feat_file, sep="\t", header=None)
            
            for ids in ohe_data.keys():
                
                flex_df_row = flex_df[flex_df[0] == ids]
                
                if not flex_df_row.empty:
                    # Take the ith feature from flex_df
                    flex_df_feat = flex_df_row.iloc[0, i+1]  # i+1 because the first column is the identifier
                    combined_data[ids]['feat'].append(flex_df_feat)
                
        except Exception as e:
            raise Exception(f"Error in extending flexibility feature from {flex_feat_file}: {e}")
    
    # Define X, y
    X = np.array([combined_data[x]['feat'] for x in combined_data.keys()])
    y = np.array([combined_data[x]['aff'] for x in combined_data.keys()])

    return X, y

In [12]:
# load DNaseI data for a single position (14 * 4 + 1 = 57) 
x,x1 = encode_data_per_position(fileprefix="bHLH_Bhlhb2_TCAAGG20NGAA_CACGTG_14_4", features=("1mer", "DNaseI"))
x

1mer data loaded from ../../../DATA/02_L2MLR/ROHS_MWORDS_1mer/bHLH_Bhlhb2_TCAAGG20NGAA_CACGTG_14_4_1mer.tsv... 

Will load the additional flex feature DNaseI


array([[ 1.   ,  0.   ,  0.   , ...,  0.   ,  1.   , -0.183],
       [ 0.   ,  1.   ,  0.   , ...,  1.   ,  0.   , -0.183],
       [ 0.   ,  1.   ,  0.   , ...,  1.   ,  0.   , -0.032],
       ...,
       [ 1.   ,  0.   ,  0.   , ...,  0.   ,  0.   , -0.033],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.015],
       [ 0.   ,  0.   ,  1.   , ...,  0.   ,  0.   , -0.033]],
      shape=(8949, 57))

In [13]:

META_FILE = "../../../METADATA/ROHS_MWORDS.csv"

def read_metadata(file_path):
    metadata = []

    with open(file_path, 'r') as file:
        for line in file:
            family, tf, motif1, motif2, length, score = line.strip().split(',')
            suffix = f"{motif1}_{motif2}_{length}_{score}"

            metadata.append({
                'family': family,
                'tf': tf,
                'suffix': suffix})
            
    return metadata


FILE_IDS = read_metadata(META_FILE)


In [14]:
def determine_flexfeat_len(fileid, features):
    
#     feat_dict = {"dinuc": ['twistDisp', 'trxDi'], "trinuc": ['DNaseI', 'NPP']}
    f_path = f"../../../DATA/02_L2MLR/ROHS_MWORDS_FLEX/{fileid}_twistDisp_0.tsv"

    
    return len(pd.read_csv(f_path, sep="\t", header=None).columns)
    

In [15]:

def nested_ridge(X, y, fileid, pos, features, outer_cv_folds=5, inner_cv_folds=5):
    outer_cv = KFold(n_splits=outer_cv_folds, shuffle=True, random_state=42)
    results = []
    alphas = np.linspace(1, 500, 1000)

    for train_index, test_index in outer_cv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Inner cross-validation to select the best lambda
        ridge_cv_model = make_pipeline(StandardScaler(with_mean=False), 
                                       
                                       RidgeCV(alphas=alphas, 
                                               cv=inner_cv_folds, 
                                               scoring='r2')
                                      )

        
        
        ridge_cv_model.fit(X_train, y_train)
        best_alpha = ridge_cv_model.named_steps['ridgecv'].alpha_

        
        # Train the model on the full training data with the best lambda
        best_ridge_model = make_pipeline(StandardScaler(with_mean=False), 
                                         RidgeCV(alphas=[best_alpha], cv=None))
        
        best_ridge_model.fit(X_train, y_train)

        # Calculate R^2, MSE, and MAE for training and test sets
        r2_train = best_ridge_model.score(X_train, y_train)
        r2_test = best_ridge_model.score(X_test, y_test)

        y_train_pred = best_ridge_model.predict(X_train)
        y_test_pred = best_ridge_model.predict(X_test)

        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)

        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)

        result = {
            'fileid': fileid,
            'pos': pos,
            'features': features,
            'r2_train': r2_train,
            'r2_test': r2_test,
            'mse_train': mse_train,
            'mse_test': mse_test,
            'mae_train': mae_train,
            'mae_test': mae_test,
            'best_alpha': best_alpha
        }
        
        results.append(result)

    return results

### L2MLR by adding each flexibility features with sequence features to each position

In [16]:
from itertools import product



ohe = ["1mer"]

features = ['DNaseI', 'NPP', 'twistDisp', 'stiffness', 'trxDi']
feature_sets = [('1mer',)] + [x for x in product(ohe, features)]



results_df_pp_nested = []


# run on each protein
for record in FILE_IDS:
    
    print(f"Running for file: {record} \n")
    file_id = f"{record['family']}_{record['tf']}_{record['suffix']}"

    
    
#     for each feature
    for fset in feature_sets:
        print(f"current feature is: {fset}...\n")

        
        
        if fset == ("1mer", ):
            print("xx")
            # X,y =  encode_data_per_position(fileprefix= file_id, features= fset)
            
            # res = nested_ridge(X = X, 
            #                 y = y, 
            #                 fileid = file_id, 
            #                 pos = "all",
            #                 features = fset,
            #                 outer_cv_folds=10, 
            #                 inner_cv_folds=5)
        
            # print(res)
            # results_df_pp_nested.append(res)
            
            
        else:
            
            dt = determine_flexfeat_len(fileid=file_id, features=fset)
            
            for x in range(dt - 1):
                print(f"Encoding data of position:  {x}")

                X,y = encode_data_per_position(fileprefix= file_id, 
                                               features= fset, 
                                               i = x)    
                print(X)
                # build and test model
                res = nested_ridge(X = X, 
                                   y = y, 
                                   fileid = file_id, 
                                   pos = x,
                                   features = fset,
                                   outer_cv_folds=10, 
                                   inner_cv_folds=5)

        
                print(res)
        
                results_df_pp_nested.append(res)

Running for file: {'family': 'bHLH', 'tf': 'Bhlhb2', 'suffix': 'TCAAGG20NGAA_CACGTG_14_4'} 

current feature is: ('1mer',)...

xx
current feature is: ('1mer', 'DNaseI')...

Encoding data of position:  0
1mer data loaded from ../../../DATA/02_L2MLR/ROHS_MWORDS_1mer/bHLH_Bhlhb2_TCAAGG20NGAA_CACGTG_14_4_1mer.tsv... 

Will load the additional flex feature DNaseI
[[ 1.     0.     0.    ...  0.     1.    -0.081]
 [ 0.     1.     0.    ...  1.     0.     0.175]
 [ 0.     1.     0.    ...  1.     0.    -0.136]
 ...
 [ 1.     0.     0.    ...  0.     0.    -0.033]
 [ 0.     0.     0.    ...  0.     0.    -0.274]
 [ 0.     0.     1.    ...  0.     0.    -0.077]]


KeyboardInterrupt: 

In [17]:
feature_sets = [('1mer', )] + [x for x in product(ohe, features)]

In [42]:
x = []

for i in results_df_pp_nested:
    for k in i:
        x.append(k)

In [45]:
pd.DataFrame(x).to_csv("../../../RESULTS/04_POSITIONAL_IMPORTANCE/MLR_L2_10fold_and_5fold_CV_position_and_features.tsv", 
                      sep="\t", index=False)

### L2MLR with all features combined for each position of the binding sites

In [15]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

def encode_data_per_position(fileprefix,
                data_dir="../../../DATA/02_L2MLR",
                features=("1mer",), 
                w=0,
                i=1):
    
    try:
        ohe_file = os.path.join(data_dir, "ROHS_MWORDS_1mer", f"{fileprefix}_1mer.tsv")
        
        ohe_df = pd.read_csv(ohe_file, header=None, sep="\t")
        ohe_data = {row[0]: {'aff': float(row[0].split('__')[1]), 
                             'feat': row.iloc[1:].tolist()} for _, row in ohe_df.iterrows()}
        
        print(f"1mer data loaded from {ohe_file}... \n")
        
    except FileNotFoundError as fe:
        raise FileNotFoundError(f"File {ohe_file} Not found: {fe}")
    
    combined_data = ohe_data.copy()
    
    if isinstance(features, tuple) and len(features) > 1:
        all_feats = {ids: [] for ids in ohe_data.keys()}

        for feature in features[1:]:
            print(f"Will load the additional flex feature {feature}")
            try:
                
                flex_feat_file = os.path.join(data_dir, "ROHS_MWORDS_FLEX", f"{fileprefix}_{feature}_{w}.tsv") 
                flex_df = pd.read_csv(flex_feat_file, sep="\t", header=None)
                
                for ids in ohe_data.keys():
                    
                    flex_df_row = flex_df[flex_df[0] == ids]
                    
                    if not flex_df_row.empty and (i+1) < len(flex_df_row.columns):
                        # Take the ith feature from flex_df
                        flex_df_feat = flex_df_row.iloc[0, i+1]  # i+1 because the first column is the identifier
                        all_feats[ids].append(flex_df_feat)
                    else:
#                         print(f"Warning! No feature is present at the given position for {feature}, append 0")
                        all_feats[ids].append(0.0)
                
            except Exception as e:
                raise Exception(f"Error in extending flexibility feature from {flex_feat_file}: {e}")

        # Convert the dictionary to a DataFrame for scaling
        all_feats_df = pd.DataFrame.from_dict(all_feats, orient='index')
        print(all_feats_df)

        # Scale the additional features using StandardScaler
        scaler = StandardScaler()
        scaled_feats = scaler.fit_transform(all_feats_df)
        print(scaled_feats)
        
        # Merge scaled features with the 1mer features
        for i, ids in enumerate(ohe_data.keys()):
            combined_data[ids]['feat'].extend(scaled_feats[i].tolist())

    # Define X, y
    X = np.array([combined_data[x]['feat'] for x in combined_data.keys()])
    y = np.array([combined_data[x]['aff'] for x in combined_data.keys()])
    
    
    return X, y

In [16]:
from itertools import product



ohe = ["1mer"]

features = ['DNaseI', 'NPP', 'twistDisp', 'stiffness', 'trxDi']
feature_sets = [('1mer', ), ("1mer", "DNaseI", 'twistDisp', 'NPP', 'stiffness', 'trxDi')]


feature_sets


[('1mer',), ('1mer', 'DNaseI', 'twistDisp', 'NPP', 'stiffness', 'trxDi')]

In [17]:
# pp_fcombined = []


# # run on each protein
# for record in FILE_IDS:
    
#     print(f"Running for file: {record} \n")

#     file_id = f"{record['family']}_{record['tf']}_{record['suffix']}"

#    # for each feature
#     for fset in feature_sets:
#         print(f"current feature is: {fset}...\n")

#         if fset == ("1mer", ):
#             X,y =  encode_data_per_position(fileprefix= file_id, features= fset)


#             res = nested_ridge(X = X, 
#                                    y = y, 
#                                    fileid = file_id, 
#                                    pos = "all",
#                                    features = fset,
#                                    outer_cv_folds=10, 
#                                    inner_cv_folds=5)

#             print(res)
#             pp_fcombined.append(res)

#         else:
#             dt = determine_flexfeat_len(fileid=file_id, features=fset)
#             for x in range(dt - 1):
#                 print(f"Encoding data of position:  {x}")

#                 X,y = encode_data_per_position(fileprefix= file_id, 
#                                                features= fset, 
#                                                i = x)

# #                 print(X)
#                 # build and test model
#                 res = nested_ridge(X = X, 
#                                    y = y, 
#                                    fileid = file_id, 
#                                    pos = x,
#                                    features = fset,
#                                    outer_cv_folds=10, 
#                                    inner_cv_folds=5)

#                 print(res)
#                 pp_fcombined.append(res)

Running for file: {'family': 'bHLH', 'tf': 'Bhlhb2', 'suffix': 'TCAAGG20NGAA_CACGTG_14_4'} 

current feature is: ('1mer',)...

1mer data loaded from ../../../DATA/02_L2MLR/ROHS_MWORDS_1mer/bHLH_Bhlhb2_TCAAGG20NGAA_CACGTG_14_4_1mer.tsv... 



KeyboardInterrupt: 

LinAlgError: SVD did not converge

In [18]:
pp_fcombined = []

# run on each protein
for record in FILE_IDS:
    
    print(f"Running for file: {record} \n")

    file_id = f"{record['family']}_{record['tf']}_{record['suffix']}"

    # for each feature
    for fset in feature_sets:
        print(f"current feature is: {fset}...\n")

        if fset == ("1mer", ):
            try:
                X, y = encode_data_per_position(fileprefix=file_id, features=fset)
                res = nested_ridge(X=X, 
                                   y=y, 
                                   fileid=file_id, 
                                   pos="all",
                                   features=fset,
                                   outer_cv_folds=10, 
                                   inner_cv_folds=5)
                print(res)
                pp_fcombined.append(res)
            except Exception as e:
                print(f"Model failed for {file_id} with feature {fset} and position all. Error: {e}")
                continue

        else:
            dt = determine_flexfeat_len(fileid=file_id, features=fset)
            for x in range(dt - 1):
                print(f"Encoding data of position: {x}")
                try:
                    X, y = encode_data_per_position(fileprefix=file_id, 
                                                    features=fset, 
                                                    i=x)
                    res = nested_ridge(X=X, 
                                       y=y, 
                                       fileid=file_id, 
                                       pos=x,
                                       features=fset,
                                       outer_cv_folds=10, 
                                       inner_cv_folds=5)
                    print(res)
                    pp_fcombined.append(res)
                except Exception as e:
                    print(f"Model failed for {file_id} with feature {fset} and position {x}. Error: {e}")
                    continue

Running for file: {'family': 'bHLH', 'tf': 'Bhlhb2', 'suffix': 'TCAAGG20NGAA_CACGTG_14_4'} 

current feature is: ('1mer',)...

1mer data loaded from ../../../DATA/02_L2MLR/ROHS_MWORDS_1mer/bHLH_Bhlhb2_TCAAGG20NGAA_CACGTG_14_4_1mer.tsv... 



KeyboardInterrupt: 

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/PAX_PAX2_TCCACG20NCAC_CACGCW_14_4_1mer.tsv... 

[{'fileid': 'PAX_PAX2_TCCACG20NCAC_CACGCW_14_4', 'pos': 'all', 'features': ('1mer',), 'r2_train': 0.46337811362924497, 'r2_test': 0.3633697424098091, 'mse_train': 0.005259034763439831, 'mse_test': 0.005900685707457106, 'mae_train': 0.05560893718451498, 'mae_test': 0.05959582391493408, 'best_alpha': 500.0}, {'fileid': 'PAX_PAX2_TCCACG20NCAC_CACGCW_14_4', 'pos': 'all', 'features': ('1mer',), 'r2_train': 0.4555161766391548, 'r2_test': 0.41945815934729935, 'mse_train': 0.005317883303614833, 'mse_test': 0.0055390656005596195, 'mae_train': 0.05565952574881725, 'mae_test': 0.0590593066656664, 'best_alpha': 500.0}, {'fileid': 'PAX_PAX2_TCCACG20NCAC_CACGCW_14_4', 'pos': 'all', 'features': ('1mer',), 'r2_train': 0.4591887740701476, 'r2_test': 0.4121602408684686, 'mse_train': 0.005338114840263855, 'mse_test': 0.0050820106194080575, 'mae_train': 0.05595

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/PAX_PAX2_TCCACG20NCAC_CACGCW_14_4_1mer.tsv... 

Will load the additional flex feature DNaseI
Will load the additional flex feature twistDisp
Will load the additional flex feature NPP
Will load the additional flex feature stiffness
Will load the additional flex feature trxDi
                                            0    1   2     3   4
CGGTCACGCTTGAC__1__147                 -0.032  5.2   8  3500  42
CCGTCACGCATGAG__0.994262392544527__202 -0.033  5.5   8  1340  43
CCGTCACGCTTGAC__0.990244375241414__142 -0.033  5.5   8  1340  43
CGGTCACGCTTGAG__0.973956139968075__134 -0.032  5.2   8  3500  42
CCGTCACGCTTGAG__0.972866052603556__134 -0.033  5.5   8  1340  43
...                                       ...  ...  ..   ...  ..
TCGTCACACATGAA__0.401178054442318__9   -0.033  5.5   8  1340  43
ATGTCATGCATGAA__0.400067538159256__9   -0.006  6.5   6  1370  42
CAGTCATGCATAAA__0.397384297698789__9   -0

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/PAX_PAX2_TCCACG20NCAC_CACGCW_14_4_1mer.tsv... 

Will load the additional flex feature DNaseI
Will load the additional flex feature twistDisp
Will load the additional flex feature NPP
Will load the additional flex feature stiffness
Will load the additional flex feature trxDi
                                            0    1   2     3   4
CGGTCACGCTTGAC__1__147                  0.194  4.4   8  1230  22
CCGTCACGCATGAG__0.994262392544527__202  0.194  4.4   8  1230  22
CCGTCACGCTTGAC__0.990244375241414__142  0.194  4.4   8  1230  22
CGGTCACGCTTGAG__0.973956139968075__134  0.194  4.4   8  1230  22
CCGTCACGCTTGAG__0.972866052603556__134  0.194  4.4   8  1230  22
...                                       ...  ...  ..   ...  ..
TCGTCACACATGAA__0.401178054442318__9    0.194  4.4   8  1230  22
ATGTCATGCATGAA__0.400067538159256__9    0.194  4.4   8  1230  22
CAGTCATGCATAAA__0.397384297698789__9    0

...

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4_1mer.tsv... 

Will load the additional flex feature DNaseI
Will load the additional flex feature twistDisp
Will load the additional flex feature NPP
Will load the additional flex feature stiffness
Will load the additional flex feature trxDi
                                          0    1   2     3   4
CCACATTCCATG__1__234                  0.040  6.5  17  1370  42
CCACATTCCACG__0.979278102202361__264  0.040  6.5  17  1370  42
CCACATTCCACA__0.977864846148202__339  0.040  6.5  17  1370  42
ACACATTCCACA__0.976404053378618__337  0.040  6.5  17  1370  42
ACACATTCCTGA__0.969586128213483__205  0.040  6.5  17  1370  42
...                                     ...  ...  ..   ...  ..
CCCCAACCCCCC__0.333522147887467__9   -0.012  5.2  13  3500  42
CCGCACCCCCCC__0.32977947390551__9    -0.077  5.5  25  1340  43
CCCCACCCCCAC__0.328826912610277__9   -0.012  5.2  13  3500

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4_1mer.tsv... 

Will load the additional flex feature DNaseI
Will load the additional flex feature twistDisp
Will load the additional flex feature NPP
Will load the additional flex feature stiffness
Will load the additional flex feature trxDi
                                          0    1   2     3   4
CCACATTCCATG__1__234                  0.134  6.5  18  1370  42
CCACATTCCACG__0.979278102202361__264  0.134  6.5  18  1370  42
CCACATTCCACA__0.977864846148202__339  0.134  6.5  18  1370  42
ACACATTCCACA__0.976404053378618__337  0.134  6.5  18  1370  42
ACACATTCCTGA__0.969586128213483__205  0.134  6.5  18  1370  42
...                                     ...  ...  ..   ...  ..
CCCCAACCCCCC__0.333522147887467__9    0.015  6.5   9  1370  42
CCGCACCCCCCC__0.32977947390551__9     0.040  6.5  17  1370  42
CCCCACCCCCAC__0.328826912610277__9    0.040  6.5  17  1370

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4_1mer.tsv... 

Will load the additional flex feature DNaseI
Will load the additional flex feature twistDisp
Will load the additional flex feature NPP
Will load the additional flex feature stiffness
Will load the additional flex feature trxDi
                                          0    1   2     3   4
CCACATTCCATG__1__234                 -0.037  3.9  12  1820   5
CCACATTCCACG__0.979278102202361__264 -0.037  3.9  12  1820   5
CCACATTCCACA__0.977864846148202__339 -0.037  3.9  12  1820   5
ACACATTCCACA__0.976404053378618__337 -0.037  3.9  12  1820   5
ACACATTCCTGA__0.969586128213483__205 -0.037  3.9  12  1820   5
...                                     ...  ...  ..   ...  ..
CCCCAACCCCCC__0.333522147887467__9   -0.032  4.2   8  1660   4
CCGCACCCCCCC__0.32977947390551__9    -0.012  5.2  13  3500  42
CCCCACCCCCAC__0.328826912610277__9   -0.012  5.2  13  3500

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4_1mer.tsv... 

Will load the additional flex feature DNaseI
Will load the additional flex feature twistDisp
Will load the additional flex feature NPP
Will load the additional flex feature stiffness
Will load the additional flex feature trxDi
                                          0    1   2     3   4
CCACATTCCATG__1__234                 -0.246  5.2   8  3500  42
CCACATTCCACG__0.979278102202361__264 -0.246  5.2   8  3500  42
CCACATTCCACA__0.977864846148202__339 -0.246  5.2   8  3500  42
ACACATTCCACA__0.976404053378618__337 -0.246  5.2   8  3500  42
ACACATTCCTGA__0.969586128213483__205 -0.057  5.2   8  3500  42
...                                     ...  ...  ..   ...  ..
CCCCAACCCCCC__0.333522147887467__9   -0.012  5.2  13  3500  42
CCGCACCCCCCC__0.32977947390551__9    -0.012  5.2  13  3500  42
CCCCACCCCCAC__0.328826912610277__9   -0.012  5.2  13  3500

1mer data loaded from /SynologyNAS/001_Thesis_work/Prj_TF_FAMILY_SPECIFIC/RESULTS/ROHS_MWORDS_1mer/TEA_TEAD3_TATAAG40NAAG_CATTCC_12_4_1mer.tsv... 

Will load the additional flex feature DNaseI
Will load the additional flex feature twistDisp
Will load the additional flex feature NPP
Will load the additional flex feature stiffness
Will load the additional flex feature trxDi
                                          0    1   2     3   4
CCACATTCCATG__1__234                  0.134  4.5  18  5500   0
CCACATTCCACG__0.979278102202361__264 -0.033  4.2   8  1660   4
CCACATTCCACA__0.977864846148202__339 -0.006  4.2   6  1660   4
ACACATTCCACA__0.976404053378618__337 -0.006  4.2   6  1660   4
ACACATTCCTGA__0.969586128213483__205  0.194  6.5   8  1370  42
...                                     ...  ...  ..   ...  ..
CCCCAACCCCCC__0.333522147887467__9   -0.012  5.2  13  3500  42
CCGCACCCCCCC__0.32977947390551__9    -0.012  5.2  13  3500  42
CCCCACCCCCAC__0.328826912610277__9    0.040  6.5  17  1370

In [40]:
# pd.DataFrame([k for k in i for i in pp_fcombined])

df = []

for i in pp_fcombined:
    for k in i:
        df.append(k)

In [41]:
df

[{'fileid': 'PAX_PAX2_TCCACG20NCAC_CACGCW_14_4',
  'pos': 'all',
  'features': ('1mer',),
  'r2_train': 0.46337811362924497,
  'r2_test': 0.3633697424098091,
  'mse_train': 0.005259034763439831,
  'mse_test': 0.005900685707457106,
  'mae_train': 0.05560893718451498,
  'mae_test': 0.05959582391493408,
  'best_alpha': 500.0},
 {'fileid': 'PAX_PAX2_TCCACG20NCAC_CACGCW_14_4',
  'pos': 'all',
  'features': ('1mer',),
  'r2_train': 0.4555161766391548,
  'r2_test': 0.41945815934729935,
  'mse_train': 0.005317883303614833,
  'mse_test': 0.0055390656005596195,
  'mae_train': 0.05565952574881725,
  'mae_test': 0.0590593066656664,
  'best_alpha': 500.0},
 {'fileid': 'PAX_PAX2_TCCACG20NCAC_CACGCW_14_4',
  'pos': 'all',
  'features': ('1mer',),
  'r2_train': 0.4591887740701476,
  'r2_test': 0.4121602408684686,
  'mse_train': 0.005338114840263855,
  'mse_test': 0.0050820106194080575,
  'mae_train': 0.05595318703920463,
  'mae_test': 0.0560057577977889,
  'best_alpha': 500.0},
 {'fileid': 'PAX_PAX2_T

In [42]:
pd.DataFrame(df).to_csv("../../../RESULTS/04_POSITIONAL_IMPORTANCE/L2MLR_FEng_combinedFeatures.csv",
                       index=False)